In [1]:
%matplotlib notebook

In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from numpy import std, pad, amax, abs
from numpy.random import normal
from numpy.fft import fftshift, ifftshift, fft2, ifft2 
from cv2 import imshow as cv2_imshow
#from google.colab.patches import cv2_imshow

In [275]:
# Filenames
filename = './pics/lenna.jpg'
test_filenames = ['./pics/barraxx.bmp', './pics/BDBOB.jpg']

# Show original image

In [4]:
original = cv.imread(filename=filename, flags=cv.IMREAD_COLOR)
cv.imshow(winname = 'original', mat = original)
cv.waitKey(0)
cv.destroyAllWindows()

# Blur + Noise

In [5]:
def noisy_gauss(image, snr=20):
      row, col, ch = image.shape
      sigma_image = std(image)
      sigma_noise = np.sqrt(sigma_image**2 * 10**(-snr/10))
      noise = normal(loc=0.0, scale=sigma_noise, size=image.shape)
      noisy = image + noise
      noisy = np.where(noisy < 0,  0, noisy)
      noisy = np.where(noisy > 255, 255, noisy)
      noisy = noisy.astype(dtype=np.uint8)
      return noisy, sigma_noise

# Wiener and Inverse Recovering

In [6]:
kernel_shape = (8, 8)
kernel = 1 / kernel_shape[0] / kernel_shape[1]
kernel *= np.ones(shape=kernel_shape)

lenna_blur = cv.blur(src=original, ksize=(kernel_shape[0], kernel_shape[1]))
lenna_blur_noise, noise_sigma = noisy_gauss(image=lenna_blur, snr=10)
cv2_imshow('lena blured and noisy',lenna_blur_noise)
cv.waitKey(0)
cv.destroyAllWindows()

In [7]:
cv.split(lenna_blur_noise)

[array([[127, 139, 117, ..., 105,  88,  90],
        [124, 111, 155, ...,  75, 100, 125],
        [144, 139, 147, ...,  76, 106, 106],
        ...,
        [ 24,  13,  39, ...,  49,  32,  51],
        [ 13,  14,  20, ...,  76,  65,  51],
        [ 12,  30,   0, ...,  45,  43,  48]], dtype=uint8),
 array([[147, 123, 140, ..., 108,  97,  96],
        [144, 120, 142, ..., 102,  94,  82],
        [126, 144, 124, ..., 104,  75, 111],
        ...,
        [  7,  27,  36, ...,  28,  39,  42],
        [ 31,  40,   0, ...,  40,  56,  47],
        [  7,   5,  23, ...,  28,  19,  62]], dtype=uint8),
 array([[133, 124, 137, ..., 101, 106, 113],
        [101, 146, 141, ...,  88,  79,  88],
        [147, 134, 116, ..., 114,  59,  77],
        ...,
        [ 47,  13,  24, ...,  31,  37,  64],
        [ 10,  41,  16, ...,  54,  71,  52],
        [ 33,  14,  47, ...,  63,  72,  35]], dtype=uint8)]

In [151]:
def center_image(large_image, center_image, debug = False):
    x_min = (large_image.shape[0] // 2) - (center_image.shape[0] // 2)
    x_max = (large_image.shape[0] // 2) + (center_image.shape[0] // 2)
    y_min = (large_image.shape[1] // 2) - (center_image.shape[1] // 2)
    y_max = (large_image.shape[1] // 2) + (center_image.shape[1] // 2)
    result = np.zeros((large_image.shape[0], large_image.shape[1]))
    result[x_min:x_max, y_min:y_max] = center_image
    if debug:
        test_list = [x_min, x_max, y_min, y_max]
        print(test_list)
    return result

kernel_padded = center_image(np.zeros((16, 16)), kernel, debug = True)
print(kernel_padded)

[4, 12, 4, 12]
[[0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.       0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.015625 0.015625 0.015625 0.015625
  0.015625 0.015625 0.015625 0.015625 0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.015625 0.015625 0.015625 0.015625
  0.015625 0.015625 0.015625 0.015625 0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.015625 0.015625 0.015625 0.015625
  0.015625 0.015625 0.015625 0.015625 0. 

In [300]:
def recover_v2(degradated_img, psf, sn=None, wiener_inverse=False, show = False):
    #paso al espacio de frecuencias la psf y la imagen degradada. El tamaño de la psf puede ser 
    #menor al tamaño de la imagen degradada.
    
    #F = G.H, F: imagen original, G: imagen degragaga, H:sistema
    channels = []
    degradated_img = degradated_img / np.max(degradated_img)
    for img_channel in cv.split(degradated_img):
        #print(img_channel.shape)
        psf_padded = center_image(img_channel, psf) ## en esta funcion se utilizará el tamaño de la imagen para 
                                                    ##crear un nuevo kernel con dicho tamaño.
        H = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(psf_padded)))
        G = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(img_channel)))
        if (wiener_inverse) and (not(sn.any() == None)):           #WIENER OPTION
            
            psd_img_channel = np.ones((img_channel.shape[0], img_channel.shape[1]))
            aux = np.fft.fft2(np.corrcoef(img_channel))
            psd_img_channel[0:aux.shape[0], 0:aux.shape[1]] = aux
            
            sn_matrix = np.zeros((img_channel.shape[0], img_channel.shape[1]))
            
            np.fill_diagonal(sn_matrix, sn[0])
            
            Sn_matrix = np.fft.fft2(sn)
            
            noise_relation = np.abs( Sn_matrix/ psd_img_channel) + 256*256
            #print(noise_relation)
            H = H + 10
            W = H.conj()/((H*H.conj()) + noise_relation)
            #W = np.where(W == 0, W, 1)
            F = np.multiply(G, W)
            f = np.abs(np.fft.ifftshift((np.fft.ifft2(np.fft.ifftshift(F)))))
            #print(f.max())
            #print(f.min())
            maximum = np.max(f)
            f = np.uint8((f/maximum)*256)
        else:                        #INVERSE_SIMPLE OPTION
            #H = np.where(H == 0, H, 1)
            F = np.divide(G, H + 10)
            f = np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(F))))
            maximum = np.max(f)
            f = np.uint8((f/maximum)*256)
        #print(f.max())
        #print(f.min())
        
        #cv2_imshow('channel', f)
        #cv.waitKey(0)
        #cv.destroyAllWindows()
        channels.append(f)
        
    if len(channels) > 1:
        clean_img = cv.merge(channels)
        
    
    return clean_img

def mse_metric(x, y):
    a = x.shape[0]
    b = x.shape[1]
    c = x.shape[2]
    x = x.reshape(1, a*b*c)
    y = y.reshape(1, a*b*c)
    return  np.mean((x - y)**2)   
    

In [301]:
noise_shape = (lenna_blur_noise.shape[0], lenna_blur_noise.shape[1])
noise = np.ones(shape=noise_shape) * noise_sigma**2

wiener_result = recover_v2(lenna_blur_noise, kernel, sn=noise, wiener_inverse=True)
inverse_simple = recover_v2(lenna_blur_noise, kernel, wiener_inverse=False)

print(f'MSE degradated {mse_metric(original, lenna_blur_noise)}')
print(f'MSE inverse_simple {mse_metric(original, inverse_simple)}')
print(f'MSE Wiener {mse_metric(original, wiener_result)}')

wiener_vs_inverse_stacked = np.hstack((lenna_blur_noise, inverse_simple, wiener_result))
cv2_imshow('degradated <LEFT>, inverse_simple <CENTER> and Wiener <RIGHT>', wiener_vs_inverse_stacked)
cv.waitKey(0)
cv.destroyAllWindows()




C:\Users\malen\Anaconda3\envs\images\lib\site-packages\ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


MSE degradated 86.77205912272136
MSE inverse_simple 89.96910603841145
MSE Wiener 86.15191141764323


In [243]:
def estimate_blur_kernel(img):
  #estimated = None
  estimated = (1/9)*np.ones((4, 4))
  return estimated

In [295]:
for name in test_filenames:
  img = cv.imread(name)
  if img.any() == None:
        print("invalid image")
  noise_shape = (img.shape[0], img.shape[1])
  # assuming noise_sigma based on lena... should change the estimated value
  # based on the new image 
  noise = np.ones(shape=noise_shape) * noise_sigma**2

  kernel = estimate_blur_kernel(img)
  wiener_result = recover_v2(img, kernel, sn=noise, wiener_inverse=True, show = False)
  inverse_result = recover_v2(img, kernel, wiener_inverse=False, show = False)
  stacked = np.hstack((wiener_result, inverse_result))
  cv2_imshow('wiener vs inverse', stacked)
  cv.waitKey(0)
  cv.destroyAllWindows()
  print('Wiener vs Inverse')

C:\Users\malen\Anaconda3\envs\images\lib\site-packages\ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


Wiener vs Inverse
Wiener vs Inverse


### Blind Deconvolution

Blind Deconvolution (Deconvolución a ciegas) consiste en un algoritmo que reconstruye la imagen sin necesidad de tener el modelo exacto de la función "PSF" (es por esto que se le da el nombre de "a ciegas"). El algortimo parte de una función PSF "a priori", o supone una forma de función PSF, pero luego la ajusta según la imagen de entrada. Para ajustar la función PSF, el algoritmo suele usar las secciones más brillantes de la imagen, que se vieron menos afectadas por el nivel de ruido.
Para la implementación, se utilizó la función "deconvblind" de Matlab, a la que se la paso la sencilla función PSF a priori como una matriz de 12x12 de unos. Con esto logro obtenerse resultados aceptables.

In [277]:
matlab_code_img = cv.imread('deconvCodeMATLAB.png')
#print(matlab_code_img)
cv.imshow('matlab code deconvblind', matlab_code_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [278]:
in_image_blind = cv.imread('image_in_deconvblind.bmp') 
out_image_blind = cv.imread('image_out_deconvblind.bmp') 
in_out_images = np.hstack((in_image_blind, out_image_blind))
cv.imshow('in image -LEFT- vs out -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()

### Motion Blur

Para simular el motion blur que degrada una imagen debido a un movimiento de una cámara al momento de tomar la foto, se realizan kernels con unos en una columna (en caso de motion vertical) o en una fila (en caso de blur horizonral).

In [281]:
kernel_h_motion = np.zeros((20, 20))
kernel_v_motion = np.zeros((20, 20))

k = 1 / 20

kernel_h_motion[2, :] = k

kernel_v_motion[:, 2] = k



#print(kernel_v_motion)
#print(kernel_h_motion)

#### Vertical Motion

In [296]:
original = cv.imread(filename=filename, flags=cv.IMREAD_COLOR)
vertical_moved = cv.filter2D(original, -1, kernel_v_motion)
noise = np.ones(shape=(256, 256)) * (noise_sigma/10)**2
#vertical_recovered = recover(vertical_moved, kernel_v_motion, sn= noise, wiener_inverse=True, show = False)
vertical_recovered = recover_v2(vertical_moved, kernel_v_motion, sn= noise, wiener_inverse=False, show = False)
in_out_images = np.hstack((original, vertical_moved, vertical_recovered))

print(f'MSE degradated {mse_metric(original, vertical_moved)}')
print(f'MSE recovered {mse_metric(original, vertical_recovered)}')


cv.imshow('original - LEFT- moved image -CENTER- vs recovered -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()

MSE degradated 86.298583984375
MSE recovered 92.0057373046875


#### Horizontal Motion

In [297]:

horizontal_moved = cv.filter2D(original, -1, kernel_h_motion)
noise = np.ones(shape=(256, 256)) * (noise_sigma/10)**2
#horizontal_recovered = recover(horizontal_moved, kernel_h_motion, sn= noise, wiener_inverse=True, show = False)
horizontal_recovered = recover_v2(horizontal_moved, kernel_h_motion, sn= noise, wiener_inverse=True, show = False)
in_out_images = np.hstack((original, horizontal_moved, horizontal_recovered))

print(f'MSE degradated {mse_metric(original, horizontal_moved)}')
print(f'MSE recovered {mse_metric(original, horizontal_recovered)}')
cv.imshow('original - LEFT- moved image -CENTER- vs recovered -RIGHT-', in_out_images)
cv.waitKey(0)
cv.destroyAllWindows()

C:\Users\malen\Anaconda3\envs\images\lib\site-packages\ipykernel_launcher.py:18: ComplexWarning: Casting complex values to real discards the imaginary part


MSE degradated 82.41682434082031
MSE recovered 106.2109375
